In [1]:
from matplotlib import pyplot as plt
import glob
import numpy as np
from PIL import Image
from tqdm import tqdm
import os
import pathlib

import cv2
import numpy as np
import PIL
from PIL import Image
from math import pi, ceil, floor
import math
from matplotlib import pyplot as plt
import random
from tqdm import tqdm
from sklearn.cluster import DBSCAN
from scipy.spatial import ConvexHull, convex_hull_plot_2d
import skimage.io
from scipy import ndimage as ndi
from statistics import mean
import glob

print("Done")

Done


In [2]:
def do_image(original, filename, R, G, B):
    thresh_lo = 157
    thresh_hi = 200
    erode_val = 10
    gauss_val = 9
    exponent = 0.18
    gauss_val2 = 3
    original = cv2.cvtColor(original, cv2.COLOR_BGR2RGB)
    r_channel, g_channel, b_channel = cv2.split(original)
    color_channels = [r_channel, g_channel, b_channel]
    alpha_channel = np.ones(b_channel.shape, dtype=b_channel.dtype) * 50 #creating a dummy alpha channel image.
    default_vals = [R, G, B]
        
    border_int = 3
    img = cv2.copyMakeBorder(cv2.cvtColor(original, cv2.COLOR_BGR2GRAY), border_int, border_int, border_int, border_int, cv2.BORDER_CONSTANT, value=255)
    
    ret, thresh = cv2.threshold(img, thresh_lo, thresh_hi, cv2.THRESH_BINARY)
    dist = cv2.distanceTransform(thresh, cv2.DIST_L2, cv2.DIST_MASK_PRECISE)
    dist = cv2.GaussianBlur(dist, (gauss_val, gauss_val), 0)
    dist = dist[border_int:img.shape[0]-border_int, border_int:img.shape[1]-border_int]
    norm_dist = cv2.normalize(dist, None, 0, 1.0, cv2.NORM_MINMAX)
    alpha_channel = cv2.bitwise_not((255 * norm_dist).astype(b_channel.dtype))
    
    kernel = np.ones((erode_val, erode_val), np.uint8)
    alpha_channel = cv2.erode(alpha_channel, kernel)
    
    img_RGBA = Image.fromarray(cv2.cvtColor(cv2.merge((r_channel, g_channel, b_channel, alpha_channel)), cv2.COLOR_BGRA2RGBA))
    
    # # flattens alpha-channel image
    # background = Image.new('RGBA', (np.shape(img_RGBA)[1], np.shape(img_RGBA)[0]), (255,255,255))
    # img_RGB = Image.fromarray(cv2.cvtColor(np.array(Image.alpha_composite(background, img_RGBA)), cv2.COLOR_RGBA2RGB))
    # img_RGB.save(filename, 'JPEG', quality=100, subsampling=0)
    
    # normal png save w/ alpha
    new_filename = filename.split("\\")[0] + "\\" + filename.split("\\")[-1].split(".")[0] + ".png"
    img_RGBA.save(new_filename)
    
    return Image.open(new_filename), alpha_channel

print("Done")

Done


In [17]:
tmp_fp = "tmp/"

try:
    for path, subdirs, files in os.walk(tmp_fp):
        for name in files:
            os.remove(pathlib.PurePath(path, name))
except OSError:
    print("Error")

dataset_paths = {
                 # 'PD': {'none': {'path': 'Dataset/PollenDataset/None', 'label': [0, 0, 0]},
                 #        'pollen': {'path': 'Dataset/PollenDataset/Pollen', 'label': [1, 0, 0]}},
                 'BA': {'none': {'path': 'Dataset/BeeAlarmed/None', 'label': [0, 0, 0]},
                        'pollen': {'path': 'Dataset/BeeAlarmed/Pollen', 'label': [1, 0, 0]},
                        'varroa': {'path': 'Dataset/BeeAlarmed/Varroa', 'label': [0, 1, 0]},
                        'wasps': {'path': 'Dataset/BeeAlarmed/Wasps', 'label': [0, 0, 1]}},
                 'YM': {'none': {'path': 'Dataset/YangModel/None', 'label': [0, 0, 0]},
                        'pollen': {'path': 'Dataset/YangModel/Pollen', 'label': [1, 0, 0]},
                        'varroa': {'path': 'Dataset/YangModel/Varroa', 'label': [0, 1, 0]}},
                }
                 # 'USU': {'none': {'path': 'Dataset/USU/None', 'label': [0, 0, 0]},
                 #         'pollen': {'path': 'Dataset/USU/Pollen', 'label': [1, 0, 0]},
                 #         'varroa': {'path': 'Dataset/USU/Varroa', 'label': [0, 1, 0]}}}
file_cnt = 0
for dataset_path in dataset_paths:
    for dataset in dataset_paths[dataset_path]:
        for path, subdirs, files in os.walk(dataset_paths[dataset_path][dataset]['path']):
            for name in files:
                file_cnt = file_cnt + 1
            
with tqdm(total=file_cnt, unit="images", desc=str("Remove BG")) as pbar:
    for dataset_path in dataset_paths:
        for dataset in dataset_paths[dataset_path]:
            for filename in glob.glob(dataset_paths[dataset_path][dataset]['path'] + '/*'):
                image = Image.open(filename)#.convert('RGB')\
                if filename.split("/")[1] == "PollenDataset":
                    R = 80
                    G = 146
                    B = 180
                    new_image, _ = do_image(np.array(image), tmp_fp + filename, R, G, B)
                elif filename.split("/")[1] == "BeeAlarmed":
                    R = 135
                    G = 226
                    B = 192
                    new_image, _ = do_image(np.array(image), tmp_fp + filename, R, G, B)
                elif filename.split("/")[1] == "YangModel":
                    break
                #new_image, _ = do_image(np.array(image).astype('uint8'), tmp_fp + filename, R, G, B)
                pbar.update(1)


print("Done")

Error


Remove BG:   4%|██▌                                                             | 370/9304 [00:12<05:03, 29.41images/s]

KeyboardInterrupt

